In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import style
from matplotlib import rc

rc('font', **{'family': 'serif',
   'serif': ['Computer Modern']})
rc('text', usetex=True)
style.use('style.file')

In [ ]:
# some functions
def get_track_quality_cut_9yr(fit_quality):
    if fit_quality <= 80:
        return 10**4.6
    elif fit_quality > 80 and fit_quality < 120:
        return np.power(10, 4.6 + (0.6/40) * (fit_quality - 80))
    elif fit_quality >= 120:
        return 10**5.2

def get_track_quality_cut_NextGen(speed):
    new_scaling = 1.27 * 0.94 # this fixes our MC issue from before
    if speed < 0.26:
        return (10**5.25)*new_scaling
    elif speed >= 0.26 and speed < 0.28:
        # return np.power(10, 4.65 + (0.6/0.02) * (speed - 0.26)) 
        return (np.power(10, 5.2 - (0.6/0.02) * (speed - 0.26)))*new_scaling
    elif speed >= 0.28:
        return (10**4.65)*new_scaling

plotting_options = {
    'xlabel': r'LineFit Speed',
    'ylabel': 'Q / PE',
    'zlabel': 'Events per year',
    'cmap': plt.cm.plasma ,
    'norm': colors.LogNorm(),
    'zlims':  (1E-5, 1E2)
}
    
speed_bins = np.linspace(0, 2, 251)
charge_bins = np.logspace(4, 7, 31)
speed_lims = [0, 0.5]
speed_bins = np.linspace(0, 2, 251)

charge_cut_vals = []
for s in speed_bins:
    charge_cut_vals.append(get_track_quality_cut_NextGen(s))


In [ ]:
# load up
cor_info = np.load('corsika_stuff_new.npz')
nugen_info = np.load('nugen_stuff_new.npz')
juliet_info = np.load('juliet_stuff_new.npz')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,5))

# corsika
cor_sum, cor_xedges, cor_yedges, cor_im = ax1.hist2d(
    x = cor_info['cor_speed'], y = cor_info['cor_charge'],
    weights = cor_info['cor_weights'], bins = [speed_bins, charge_bins],
    cmap = plotting_options['cmap'], norm=colors.LogNorm()
)
ax1.set_title(r"Atmos. $\mu$")
cor_cbar = plt.colorbar(cor_im, ax=ax1)

# nugen
nugen_sum, nugen_xedges, nugen_yedges, nugen_im = ax2.hist2d(
    x = nugen_info['nugen_speed'], y = nugen_info['nugen_charge'], 
    weights = nugen_info['nugen_atmo_weights'], bins = [speed_bins, charge_bins],
    cmap = plotting_options['cmap'], norm=colors.LogNorm()
)
ax2.set_title(r"Atmos. $\nu$")
nugen_cbar = plt.colorbar(nugen_im, ax=ax2)


# juliet
ehe_sum, ehe_xedges, ehe_yedges, ehe_im = ax3.hist2d(
    x = juliet_info['ehe_speed'], y = juliet_info['ehe_charge'], 
    weights = juliet_info['ehe_weights'], bins = [speed_bins, charge_bins],
    cmap = plotting_options['cmap'], norm=colors.LogNorm()
)
norm2 = colors.LogNorm(vmin=ehe_im.norm.vmin, vmax=ehe_im.norm.vmax)
ax3.set_title(r"Cosmogenic $\nu$")
ehe_cbar = plt.colorbar(ehe_im, ax=ax3)

ax1.text(0.019, 5E6, 'IceCube Preliminary', color='red', fontsize=20)
# decorate all the stuff with titles, blah blah
for a in [ax1, ax2, ax3]: # axes
    a.set_yscale('log')
    a.set_xlabel(plotting_options['xlabel'])
    a.set_ylabel(plotting_options['ylabel'])
    a.set_xlim(speed_lims)
    a.plot(speed_bins, charge_cut_vals, color='red', lw=3)
for i in [cor_im, nugen_im, ehe_im]: # im objects
    i.set_clim(*plotting_options['zlims'])
for c in [cor_cbar, nugen_cbar, ehe_cbar]: # cbars
    c.set_label(plotting_options['zlabel'])

fig.tight_layout(w_pad=2)
version='new'
fig.savefig('hist2d_q_speed_{}.png'.format(version), dpi=300)

